In [8]:
from sklearn.metrics import roc_auc_score,accuracy_score,f1_score,precision_score,recall_score    
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
import lightgbm as lgb
import pandas as pd
import numpy as np
import os
import sys
import json
import joblib
import pickle
import time
import cma
from cma.fitness_functions import elli
from cma.optimization_tools import EvalParallel2
import warnings
warnings.filterwarnings('ignore')
import time
from datetime import datetime
from scipy.spatial import distance

cmace_dir = '.'
cmace_datadir = f'{cmace_dir}/datasets'
cmace_modeldir = f'{cmace_dir}/models'
datasets_to_run  = ['heloc','compas','wine','shop2']
models_to_run = ['DT','AB','RF','MLP', 'SVM', 'LR']
dataset_name = datasets_to_run[0]
model_name = models_to_run[0]
decision_threshold = 0.5
distance_definition = 'L2'


def load_model(cmace_modeldir,dataset_name,model_name):
    model = joblib.load(f'{cmace_modeldir}/{model_name}_{dataset_name}.model')
    return model

def load_dataset(cmace_modeldir,dataset_name,model_name):
    train_data = pd.read_csv(f'{cmace_datadir}/cf_{dataset_name}_data_train.csv')
    test_data = pd.read_csv(f'{cmace_datadir}/cf_{dataset_name}_{model_name}_test.csv')
    return train_data, test_data

def distance_function(x):
    if distance_definition == 'L2':
        return np.linalg.norm(x, ord=2)
    elif distance_definition == 'L1':
        return np.linalg.norm(x, ord=1)
    else:
        return np.linalg.norm(x, ord=1)

def warm_starting(instance_feature,counterfactual_features,counterfactual_probabilities,distance_definition):
    counterfactual_perturbations = counterfactual_features - instance_feature
    counterfactual_dists = np.linalg.norm(counterfactual_perturbations,ord=2,axis=1)
    obs_err = abs(counterfactual_probabilities-decision_threshold)
    proba_min = obs_err.min()
    proba_max = obs_err.max()
    counterfactual_weights = np.exp(-(obs_err/(proba_max-proba_min))**2)
    optimal_initialization = counterfactual_weights.dot(counterfactual_perturbations)/np.sum(counterfactual_weights)        
    covariance = np.multiply(counterfactual_weights,(counterfactual_perturbations-optimal_initialization).T).dot((counterfactual_perturbations-optimal_initialization))
    if distance_definition == 'L2':
        max_dist = np.max(counterfactual_dists)
    elif distance_definition == 'L1':
        max_dist = np.max(np.linalg.norm(counterfactual_perturbations,ord=1,axis=1))
    else:
        max_dist = np.max(np.linalg.norm(counterfactual_perturbations,ord=1,axis=1))

    return optimal_initialization, covariance, max_dist

def conterfactul_generation(model,x_instance,y_instance,initialization,max_dist):  
    def fun(x):
        global x_instance,y_instance
        x_new = x_instance + x
        y_new = model.predict(x_new.reshape(1,-1))[0]
        if y_new != y_instance:
            return np.linalg.norm(x,ord=2) #distance_function(x)
        else:
            return max_dist + np.linalg.norm(x,ord=2) #distance_function(x)
    def constraints(x):
        return [distance_function(x) - max_dist]
    cfun = cma.ConstrainedFitnessAL(fun, constraints)
    x0 = initialization
    variance = covariance.mean()
    es = cma.CMAEvolutionStrategy(x0, variance, {'maxiter':1000}).optimize(cfun,n_jobs=8,maxfun=5000,verb_disp=None)
    x_optimality = es.result.xfavorite
    return x_optimality


model = load_model(cmace_modeldir,dataset_name,model_name)
train_data, test_data = load_dataset(cmace_modeldir,dataset_name,model_name)
features = test_data.columns[:-1]
label = test_data.columns[-1]

y_test_preds = model.predict(test_data[features])
y_test_probas = model.predict_proba(test_data[features])[:,1]

y_train_preds = model.predict(train_data[features])
y_train_probas = model.predict_proba(train_data[features])[:,1]

cf_dists = []
for index, row in test_data[features].iterrows():
    x_instance = row.values
    y_instance = y_test_preds[index]
    ind = np.where(y_train_preds!=y_instance)[0]
    train_features = train_data[features].values[ind]
    train_probabilities = y_train_probas[ind]

    initialization, covariance, max_dist = warm_starting(x_instance,train_features,train_probabilities,distance_definition)

    def fun(x):
        x_new = x_instance + x
        y_new = model.predict(x_new.reshape(1,-1))[0]
        if y_new != y_instance:
            return distance_function(x)
        else:
            return max_dist + distance_function(x)
    def constraints(x):
        return [distance_function(x) - max_dist]
    cfun = cma.ConstrainedFitnessAL(fun, constraints)
    x0 = initialization
    variance = covariance.mean()

    es = cma.CMAEvolutionStrategy(x0, variance, {'maxiter':1000}).optimize(cfun,n_jobs=8,maxfun=5000,verb_disp=None)
    x_optimality = es.result.xfavorite            

    x_norm1 = np.linalg.norm(x_optimality,ord=1)
    x_norm2 = np.linalg.norm(x_optimality,ord=2)

    x_counterfactual = x_instance + x_optimality
    y_pred = model.predict(x_counterfactual.reshape(1,-1))[0]
    y_proba = model.predict_proba(x_counterfactual.reshape(1,-1))[:,1][0]
    print(index,x_norm2,y_test_preds[index],y_pred,y_test_probas[index],y_proba)
    cf_dists.append([dataset_name,model_name,index,x_norm2,y_test_preds[index],y_pred,y_test_probas[index],y_proba,x_optimality])
    cf_results = pd.DataFrame(cf_dists,columns=['dataset_name','model_name','index','x_norm2','y_base','y_pred','y_base_proba','y_proba','optimal_perturbation'])
    cf_results.to_csv('conterfactul_generation_results.csv',index=False)

(6_w,13)-aCMA-ES (mu_w=4.0,w_1=38%) in dimension 23 (seed=343398, Wed May  8 18:22:36 2024)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     13 7.341889106330487e+00 1.0e+00 1.74e+00  2e+00  2e+00 0:00.0
    2     26 6.360059345220137e+00 1.1e+00 1.60e+00  2e+00  2e+00 0:00.0
    3     39 7.038357320124859e+00 1.1e+00 1.49e+00  1e+00  1e+00 0:00.1
  100   1300 7.119430010882823e-02 1.6e+00 1.65e-02  1e-02  2e-02 0:01.6
  200   2600 2.170014360870088e-02 2.9e+00 9.71e-04  3e-04  8e-04 0:03.2
  300   3900 2.088970475475722e-02 7.1e+00 1.58e-04  3e-05  1e-04 0:04.8
0 0.02084885243825577 0 1 0.43075615972812237 0.603082851637765
